In [2]:
import sys
sys.path.insert(0, 'utils')
from matplotlib.font_manager import FontProperties
import pandas as pd
import numpy as np
import seaborn as sns
import scipy.stats as st
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
# from sklearn.svm import LinearSVC
# from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
import sklearn.metrics as metrics
# from imblearn.over_sampling import SMOTE
import ds_charts as ds
from ds_charts import plot_evaluation_results
import itertools as it
import timeit
import autosklearn.classification

In [4]:
suffix = ''
dataset = 'aq/aq_prov_base'
file_name = '0'
data_folder = f'data/{dataset}/'
# image_basefile = f'ecdc_covid_{dataset}'
# image_dir = f'images/{image_basefile}/{suffix}/'


data = pd.read_csv(f'{data_folder}{file_name}{suffix}.csv',parse_dates=True, infer_datetime_format=True)

data = data.drop(columns=['current_date'])

In [8]:
# Tratamento de missing values
# variable_types = ds.get_variable_types(data)
# inverted_variable_types = {val: key for key in variable_types for val in variable_types[key]}
# missing_list = data.isna().sum()[data.isna().sum() != 0].index.to_list()
# print(data.isna().sum()[data.isna().sum() != 0])
# missing_types = [(i,inverted_variable_types[i]) for i in missing_list]
# missing_types

# Fill deaths and cases with 0 for MVs
# data['cases'] = data['cases'].fillna(0)
# data['deaths'] = data['deaths'].fillna(0)
# data['ratio'] = data['ratio'].fillna(0)
# data['cases_per_100k'] = data['cases_per_100k'].fillna(0)
# Drop rows where there is no high_risk (class)
# data = data.dropna(subset=['high_risk_2w','avg_2weeks'])
# data = data.dropna(subset=['renewable_percentage'])
data = data.drop(columns=['Distance','Distance_Prov'])
# data = data.dropna(subset=['last_year_temp','one_year_diff'])


# Preprocessamento - encoding de strings
# le = LabelEncoder()
# # data['city'] = le.fit_transform(data['city'])
# data['country'] = le.fit_transform(data['country'])
# # data['description'] = le.fit_transform(data['description'])
# # data['weapon'] = le.fit_transform(data['weapon'])
# # data['district'] = le.fit_transform(data['district'])
# # data['neighborhood'] = le.fit_transform(data['neighborhood'])

# data['high_risk_2w'] = data['high_risk_2w'].astype(bool)

print(data.isna().sum()[data.isna().sum() != 0])


Series([], dtype: int64)


In [9]:
data.dtypes

ALARM           int64
CO_Mean       float64
CO_Min        float64
CO_Max        float64
CO_Std        float64
NO2_Mean      float64
NO2_Min       float64
NO2_Max       float64
NO2_Std       float64
O3_Mean       float64
O3_Min        float64
O3_Max        float64
O3_Std        float64
PM2.5_Mean    float64
PM2.5_Min     float64
PM2.5_Max     float64
PM2.5_Std     float64
PM10_Mean     float64
PM10_Min      float64
PM10_Max      float64
PM10_Std      float64
SO2_Mean      float64
SO2_Min       float64
SO2_Max       float64
SO2_Std       float64
GbCity          int64
dtype: object

In [46]:
# O auto-sklearn apenas aceita valores numéricos
target = 'ALARM'
y: np.ndarray = data.pop(target).values
X: np.ndarray = data.values
labels: np.ndarray = pd.unique(y)

trnX, tstX, trnY, tstY = train_test_split(X, y, stratify = y, test_size= 0.3, random_state = 0)
cls = autosklearn.classification.AutoSklearnClassifier(seed = 0, n_jobs=2)
start_time = timeit.default_timer()
cls.fit(trnX, trnY)
print(timeit.default_timer() - start_time)
start_time = timeit.default_timer()
predictions = cls.predict(tstX)
print(timeit.default_timer() - start_time)

3612.413715246
223.72186156099997


In [47]:
def get_scores_fold(y_train,y_prediction,store,model):
    store[model]['precision'].append(metrics.precision_score(y_train,y_prediction))
    store[model]['recall'].append(metrics.recall_score(y_train,y_prediction))
    store[model]['accuracy'].append(metrics.accuracy_score(y_train,y_prediction))
    store[model]['auc'].append(metrics.roc_auc_score(y_train,y_prediction))
    store[model]['F1'].append(metrics.f1_score(y_train,y_prediction))
    return store

In [48]:
model_scores = {
    # 'NB':  {'precision': [], 'recall' : [], 'accuracy': [], 'auc': []},
    # 'KNN': {'precision': [], 'recall' : [], 'accuracy': [], 'auc': []},
    # 'DT':  {'precision': [], 'recall' : [], 'accuracy': [], 'auc': []},
    'RF':  {'precision': [], 'recall' : [], 'accuracy': [], 'auc': [], 'F1': []},
    'GB':  {'precision': [], 'recall' : [], 'accuracy': [], 'auc': [], 'F1': []},
    'AS':  {'precision': [], 'recall' : [], 'accuracy': [], 'auc': [], 'F1': []},
}

get_scores_fold(tstY,predictions,model_scores,'AS')

print(model_scores['AS']['precision'])
print(model_scores['AS']['recall'])
print(model_scores['AS']['accuracy'])
print(model_scores['AS']['auc'])
print(model_scores['AS']['F1'])

{'RF': {'precision': [], 'recall': [], 'accuracy': [], 'auc': [], 'F1': []},
 'GB': {'precision': [], 'recall': [], 'accuracy': [], 'auc': [], 'F1': []},
 'AS': {'precision': [0.9493638676844783],
  'recall': [0.9341512268402604],
  'accuracy': [0.930672268907563],
  'auc': [0.9298096958171339],
  'F1': [0.941696113074205]}}

In [49]:
# picklestring = f"{data_folder}_{file_name}".replace('/','_')
# import pickle
# with open(f'askl_{picklestring}_classifier.p', 'wb') as handle:
#     pickle.dump(cls, handle, protocol=pickle.HIGHEST_PROTOCOL)
# with open(f'askl_{picklestring}_predictions.p', 'wb') as handle:
#     pickle.dump(predictions, handle, protocol=pickle.HIGHEST_PROTOCOL)

# cls = pd.read_pickle("askl_base_classifier_mdai.p")

# cls.leaderboard(detailed=True, ensemble_only=False).to_csv(f"{data_folder}{file_name}_autoskl.csv")
# print(cls.sprint_statistics())